# Natural Language Processing 
Natural Language Processing (or NLP for short) is a discipline in computing that deals with the communication between natural (human) languages and computer languages. A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how computers can understand and/or process natural/human languages. 


## Encoding Text
As we know machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to numeric values that our models can understand. There are many different ways of doing this and we will look at a few examples below. 

```I thought the movie was going to be bad, but it was actually amazing!```

```I thought the movie was going to be amazing, but it was actually bad!```

Although these two setences are very similar we know that they have very different meanings. This is because of the **ordering** of words, a very important property of textual data.

### Bag of Words
The first and simplest way to encode our data is to use something called **bag of words**. This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency. Have a look at the python function below that encodes a string of text into bag of words. 

In [3]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
    global word_encoding
    
    words = text.lower().split(" ")  # create a list of all of the words in the text
    bag = {}  # stores all of the encodings and their frequency
    for word in words:
        if word in vocab:
            encoding = vocab[word]  # get encoding from vocab
        else:
            vocab[word] = word_encoding
            encoding = word_encoding
            word_encoding += 1
                
        if encoding in bag:
            bag[encoding] += 1
        else:
            bag[encoding] = 1
    return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


This isn't really the way we would do this in practice. Notice that we've lost the order in which words appear. In fact, let's look at how this encoding works for the two sentences we showed above.


In [4]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


### Integer Encoding
The next technique we will look at is called **integer encoding**. This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words. This should hopefully fix the problem we saw before were we lost the order of words.

In [6]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
    global word_encoding
    words = text.lower().split(" ") 
    encoding = []  
    for word in words:
        if word in vocab:
            code = vocab[word]  
            encoding.append(code) 
        else:
            vocab[word] = word_encoding
            encoding.append(word_encoding)
            word_encoding += 1
    return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [7]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


Ideally when we encode words, we would like similar words to have similar labels and different words to have very different labels. For example, the words happy and joyful should probably have very similar labels so we can determine that they are similar. While words like horrible and amazing should probably have very different labels. The method we looked at above won't be able to do something like this for us. This could mean that the model will have a very difficult time determing if two words are similar or not which could result in some pretty drastic performace impacts.

### Word Embeddings
This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques word embeddings are learned by looking at many different training examples. You can add what's called an *embedding layer* to the beggining of your model and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pretrained embedding layers.


## Recurrent Neural Networks (RNN's)
Up until this point we have been using something called **feed-forward** neural networks. This simply means that all our data is fed forwards (all at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exaclty what a recurrent neural network is designed to do. When we say recurrent neural network all we really mean is a network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.

Let's have a look at what a recurrent layer might look like.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Let's define what all these variables stand for before we get into the explination.

**h<sub>t</sub>** output at time t

**x<sub>t</sub>** input at time t

**A** Recurrent Layer (loop)

What this diagram is trying to illustrate is that a recurrent layer processes words or input one at a time in a combination with the output from the previous iteration. So, as we progress further in the input sequence, we build a more complex understanding of the text as a whole.

What we've just looked at is called a **simple RNN layer**. It can be effective at processing shorter sequences of text for simple problems but has many downfalls associated with it. One of them being the fact that as text sequences get longer it gets increasingly difficult for the network to understand the text properly.



## LSTM
The layer we dicussed in depth above was called a *simpleRNN*. However, there does exist some other recurrent layers (layers that contain a loop) that work much better than a simple RNN layer. The one we will talk about here is called LSTM (Long Short-Term Memory). This layer works very similarily to the simpleRNN layer but adds a way to access inputs from any timestep in the past. Whereas in our simple RNN layer input from previous timestamps gradually disappeared as we got further through the input. With a LSTM we have a long-term memory data structure storing all the previously seen inputs as well as when we saw them. This allows for us to access any previous value we want at any point in time. This adds to the complexity of our network and allows it to discover more useful relationships between inputs and when they appear. 

## Sentiment Analysis
And now time to see a recurrent neural network in action. For this example, we are going to do something called sentiment analysis.

The formal definition of this term from Wikipedia is as follows:

*the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.*

The example we’ll use here is classifying movie reviews as either postive, negative or neutral.

*This guide is based on the following tensorflow tutorial: https://www.tensorflow.org/tutorials/text/text_classification_rnn*


### Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.
 

In [1]:
from keras.datasets import imdb
from keras.utils import pad_sequences
import tensorflow as tf
import os
import numpy
from tensorflow import keras
import numpy as np

VOCAB_SIZE = 88584
MAX_LEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [3]:
display(len(train_data[1]))
display(len(train_data[0]))

189

218

### More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

In [4]:
train_data = pad_sequences(train_data, MAX_LEN)
test_data = pad_sequences(test_data, MAX_LEN)

In [5]:
display(len(train_data[0]))
display(len(train_data[1]))

250

250

## Creating the model
Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer.

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32), # 32 dimensions for each word
    tf.keras.layers.Dense(1, activation='sigmoid') # To predict sentiment 
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


## Training

In [8]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 37s 55ms/step - loss: 0.4594 - acc: 0.7718 - val_loss: 0.3121 - val_acc: 0.8714
Epoch 2/10
625/625 [==============================] - 15s 24ms/step - loss: 0.2675 - acc: 0.8964 - val_loss: 0.2860 - val_acc: 0.8844
Epoch 3/10
625/625 [==============================] - 11s 18ms/step - loss: 0.2010 - acc: 0.9255 - val_loss: 0.2920 - val_acc: 0.8742
Epoch 4/10
625/625 [==============================] - 11s 18ms/step - loss: 0.1598 - acc: 0.9441 - val_loss: 0.2966 - val_acc: 0.8742
Epoch 5/10
625/625 [==============================] - 10s 15ms/step - loss: 0.1357 - acc: 0.9542 - val_loss: 0.3092 - val_acc: 0.8768
Epoch 6/10
625/625 [==============================] - 10s 15ms/step - loss: 0.1110 - acc: 0.9614 - val_loss: 0.3155 - val_acc: 0.8680
Epoch 7/10
625/625 [==============================] - 9s 15ms/step - loss: 0.0937 - acc: 0.9710 - val_loss: 0.3324 - val_acc: 0.8856
Epoch 8/10
625/625 [==============================] - 10s 16ms/

In [9]:
results = model.evaluate(test_data, test_labels)
display(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.5003 - acc: 0.8497


[0.5003370642662048, 0.8497200012207031]

## Predictions
Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.

In [10]:
word_idx = imdb.get_word_index()

def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_idx[word] if word in word_idx else 0 for word in tokens]
    return pad_sequences([tokens], MAX_LEN)[0]

text = "that movie was just amazing, amazingly terrible"
encoded = encode_text(text)
display(encoded)

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [11]:
encoded

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [12]:
## Decode function

reverse_word_idx = {value: key for (key, value) in word_idx.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_idx[num] + " "
    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing amazingly terrible


In [13]:
def predict(text):
    encoded_txt = encode_text(text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_txt
    result = model.predict(pred)
    display(result[0])
    
    if result[0] > 0.70:
        print("positive")
    else:
        print("negative")

positive_review = "Amazing movie, everyone did a fantastic job!"
predict(positive_review)

negative_review = "Amazing movie, never knew a movie could be that bad!"
predict(negative_review)

1/1 [==============================] - 0s 187ms/step


array([0.73842233], dtype=float32)

positive
1/1 [==============================] - 0s 14ms/step


array([0.7196801], dtype=float32)

positive


## RNN Play Generator

We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*

## Dataset

In [14]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

## Reading

In [15]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print("Length of text is : {}".format(len(text)))

Length of text is : 1115394


In [16]:
display(text[:50])

'First Citizen:\nBefore we proceed any further, hear'

## Encoding
As string isn't encoded, we need to encode it

In [17]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [18]:
display(text_as_int[:50])

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56])

## Decoding
Integer to string coversion

In [19]:
def int_to_text(integers):
    try:
        integers = integers.numpy()
    except:
        pass
    return "".join(idx2char[integers])

print(int_to_text(text_as_int[:50]))

First Citizen:
Before we proceed any further, hear


## Creating training example

We need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [20]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [21]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [22]:
def split_input_target(chunk):
    input_text = chunk[:-1] # hell
    target_text = chunk[1:] # ello
    return input_text, target_text

dataset = sequences.map(split_input_target) # apply on every entry

In [23]:
for x, y in dataset.take(2):
    print("Input: \n")
    display(int_to_text(x))
    print("\nOutput:")
    display(int_to_text(y))

Input: 



'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'


Output:


'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

Input: 



'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '


Output:


're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'

In [24]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [25]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, 
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## Creating a loss function
This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch. 

In [27]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)
    display(example_batch_predictions.shape)  # batch_size, sequence_length, vocab_size

TensorShape([64, 100, 65])

In [28]:
print(len(example_batch_predictions))
print(example_batch_predictions[0])
pred = example_batch_predictions[0]

64
tf.Tensor(
[[-1.4138003e-03  7.2786738e-03  2.1422342e-03 ... -1.1523946e-03
   3.6591140e-03 -3.4696646e-03]
 [-2.6598424e-03 -2.6801992e-03 -4.0187775e-03 ...  3.9131213e-03
   9.3807876e-03  2.7170673e-03]
 [-3.5878513e-03 -4.2829051e-04 -8.6737127e-04 ...  7.4127386e-03
   4.8838607e-03  2.0067964e-05]
 ...
 [-7.9573103e-05  1.3600210e-02  6.1783371e-03 ...  4.1667977e-03
   8.8538509e-03  1.6807641e-03]
 [-2.3483145e-03  1.3609552e-02  5.1058158e-03 ...  2.3876666e-03
   6.2435553e-03  1.2445854e-03]
 [ 6.0152262e-05  1.6984068e-02  4.3690484e-03 ... -1.8015655e-03
   9.7164093e-03  2.8589979e-04]], shape=(100, 65), dtype=float32)


In [29]:
sampled_idx = tf.random.categorical(pred, num_samples=1)

sampled_idx = np.reshape(sampled_idx, (1, -1))[0]
predicted_chars = int_to_text(sampled_idx)

display(predicted_chars)

"GxGTeKq&mrcFT-jaf?qLzyfmiBF;'L3Xpwu3Rbgyv$RdgB,mwQVcwY'tX!HGF,WDH?$WTK'llUPa$?gbMeiSfhiteQqT?KPL$qiJ"

In [30]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

## Compiling the model

Assume our problem to be a classification problem to predict possibility of next letter

In [31]:
model.compile(optimizer="adam", loss=loss)

## Creating checkpoints

Sace checkpoints as it trains so we can load model from a checkpoint and continue training it

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Training

In [33]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 6s 26ms/step - loss: 2.6363
Epoch 2/40
172/172 [==============================] - 5s 24ms/step - loss: 1.9491
Epoch 3/40
172/172 [==============================] - 5s 24ms/step - loss: 1.6900
Epoch 4/40
172/172 [==============================] - 5s 24ms/step - loss: 1.5419
Epoch 5/40
172/172 [==============================] - 5s 24ms/step - loss: 1.4509
Epoch 6/40
172/172 [==============================] - 5s 24ms/step - loss: 1.3880
Epoch 7/40
172/172 [==============================] - 5s 24ms/step - loss: 1.3399
Epoch 8/40
172/172 [==============================] - 5s 24ms/step - loss: 1.2994
Epoch 9/40
172/172 [==============================] - 5s 24ms/step - loss: 1.2624
Epoch 10/40
172/172 [==============================] - 5s 24ms/step - loss: 1.2268
Epoch 11/40
172/172 [==============================] - 5s 24ms/step - loss: 1.1914
Epoch 12/40
172/172 [==============================] - 5s 24ms/step - loss: 1.1559
Epoch 13/40
1

## Loading the model

In [35]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [36]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generating text 

In [39]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)
    # Number of characters to generate
    num_generate = 800

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [42]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string:  Is this the end?


Is this the end?

BAPTISTA:
Now, by the jealous play may she never winnit?

BENVOLIO:
Go, broad with my true knees a tongue to purge
That she shall have the more of or love what's fire?

PAULINA:
I pray now, sit and speak again.

ROMEO:
I stay but grieve,
As if mine ears against the hidam I might venture
A shrew of the foresain of the
heir a hopeding patient cheeks, or the war
she-picked darkeness, and presemption,
Which God is more than this,--they do most kneel, and durst not on
the note, he had for revenges with
Your enemies and employ at first,
Eiends the woeful good company, as we last
A fortine egal on: and all the power incress'd
As when he flower, not light-fected for a Valician:
An old ambiricious mouse, because thereon
Have the patience grief I am for you.

HASTINGS:
O, what's the matter? You, m


## Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Classification with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_classification_rnn.
3. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
4. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.